# Pokemon GMM

In [ ]:
import numpy as np
from scipy.stats import multivariate_normal
import pandas as pd
import json

# Pre-defined functions and values
"It's a surprise tool that'll help us later!"

In [29]:
# constants
EPSILON: float = 1e-12
DELTA: float = 1e-9

# round10k(num) rounds num to the nearest 10,000s place (up to 4 decimal places)
def round10k(num):
    return float(int(num*10000))/10000

Load in our JSON here

In [3]:
# Load in JSON file
filepath = "pokemon_data.json"
with open(filepath, 'r') as file:
    data = json.load(file)

print(data)

{'Bulbasaur': {'weight': 6.9, 'weight_stddev': 0.8625, 'height': 0.7, 'height_stddev': 0.0875}, 'Ivysaur': {'weight': 13.0, 'weight_stddev': 1.625, 'height': 1.0, 'height_stddev': 0.125}, 'Venusaur': {'weight': 100.0, 'weight_stddev': 12.5, 'height': 2.0, 'height_stddev': 0.25}, 'Charmander': {'weight': 8.5, 'weight_stddev': 1.0625, 'height': 0.6, 'height_stddev': 0.075}, 'Charmeleon': {'weight': 19.0, 'weight_stddev': 2.375, 'height': 1.1, 'height_stddev': 0.1375}, 'Charizard': {'weight': 90.5, 'weight_stddev': 11.3125, 'height': 1.7, 'height_stddev': 0.2125}, 'Squirtle': {'weight': 9.0, 'weight_stddev': 1.125, 'height': 0.5, 'height_stddev': 0.0625}, 'Wartortle': {'weight': 22.5, 'weight_stddev': 2.8125, 'height': 1.0, 'height_stddev': 0.125}, 'Blastoise': {'weight': 85.5, 'weight_stddev': 10.6875, 'height': 1.6, 'height_stddev': 0.2}, 'Caterpie': {'weight': 2.9, 'weight_stddev': 0.3625, 'height': 0.3, 'height_stddev': 0.0375}, 'Metapod': {'weight': 9.9, 'weight_stddev': 1.2375, 'hei

Generate a list of all of the Pokemon in the JSON

In [5]:
filepath = "pokemon_data.json"
with open(filepath, 'r') as file:
    data = json.load(file)
x = 1
for key in data:
    print(x, key, str(data[key]["weight"])+" kg")
    x+=1

1 Bulbasaur 6.9 kg
2 Ivysaur 13.0 kg
3 Venusaur 100.0 kg
4 Charmander 8.5 kg
5 Charmeleon 19.0 kg
6 Charizard 90.5 kg
7 Squirtle 9.0 kg
8 Wartortle 22.5 kg
9 Blastoise 85.5 kg
10 Caterpie 2.9 kg
11 Metapod 9.9 kg
12 Butterfree 32.0 kg
13 Weedle 3.2 kg
14 Kakuna 10.0 kg
15 Beedrill 29.5 kg
16 Pidgey 1.8 kg
17 Pidgeotto 30.0 kg
18 Pidgeot 39.5 kg
19 Rattata 3.5 kg
20 Raticate 18.5 kg
21 Spearow 2.0 kg
22 Fearow 38.0 kg
23 Ekans 6.9 kg
24 Arbok 65.0 kg
25 Pikachu 6.0 kg
26 Raichu 30.0 kg
27 Sandshrew 12.0 kg
28 Sandslash 29.5 kg
29 NidoranF 7.0 kg
30 Nidorina 20.0 kg
31 Nidoqueen 60.0 kg
32 NidoranM 9.0 kg
33 Nidorino 19.5 kg
34 Nidoking 62.0 kg
35 Clefairy 7.5 kg
36 Clefable 40.0 kg
37 Vulpix 9.9 kg
38 Ninetales 19.9 kg
39 Jigglypuff 5.5 kg
40 Wigglytuff 12.0 kg
41 Zubat 7.5 kg
42 Golbat 55.0 kg
43 Oddish 5.4 kg
44 Gloom 8.6 kg
45 Vileplume 18.6 kg
46 Paras 5.4 kg
47 Parasect 29.5 kg
48 Venonat 30.0 kg
49 Venomoth 12.5 kg
50 Diglett 0.8 kg
51 Dugtrio 33.3 kg
52 Meowth 4.2 kg
53 Persian 3

In [16]:
df = pd.DataFrame(data).T
print(df)

            weight  weight_stddev  height  height_stddev
Bulbasaur      6.9         0.8625     0.7         0.0875
Ivysaur       13.0         1.6250     1.0         0.1250
Venusaur     100.0        12.5000     2.0         0.2500
Charmander     8.5         1.0625     0.6         0.0750
Charmeleon    19.0         2.3750     1.1         0.1375
...            ...            ...     ...            ...
Dratini        3.3         0.4125     1.8         0.2250
Dragonair     16.5         2.0625     4.0         0.5000
Dragonite    210.0        26.2500     2.2         0.2750
Mewtwo       122.0        15.2500     2.0         0.2500
Mew            4.0         0.5000     0.4         0.0500

[151 rows x 4 columns]


Let's find the heaviest and lightest Pokemon, aswell as the tallest and shortest Pokemon. Our random values will be generated within the range [0.25* lightest, 2.5*heaviest] for weight and the equivalent range for height

In [46]:
lightest = 0.25*df["weight"].min()
heaviest = 2.5*df["weight"].max()
shortest = 0.25*df["height"].min()
tallest = 2.5*df["height"].max()

rand = np.random.rand()

print(f"The shortest Pokemon is {shortest} meters tall.")
print(f"The tallest Pokemon is {tallest} meters tall.")
print(f"The lightest Pokemon is {lightest} kg.")
print(f"The heaviest Pokemon is {heaviest} kg.")

val = rand*(heaviest-lightest)+lightest
print(f"Sample weight: {round10k(val)} kg")

The shortest Pokemon is 0.05 meters tall.
The tallest Pokemon is 22.0 meters tall.
The lightest Pokemon is 0.025 kg.
The heaviest Pokemon is 1150.0 kg.
Sample weight: 113.3376 kg
